Rahmanda Afebrio Yuris Soesatyo - Chapter 15:,Processing Sequences Using RNNs and CNNs

1. Recurrent Neurons and Sequence Types

Recurrent Neural Network (RNN) memperkenalkan koneksi rekuren yang memungkinkan sebuah neuron memanfaatkan tidak hanya input saat ini
𝑥
(
𝑡
)
x(t), tetapi juga state atau output dari langkah waktu sebelumnya
𝑦
(
𝑡
−
1
)
y(t−1). Dengan mekanisme ini, keluaran pada waktu
𝑡
t dipengaruhi oleh seluruh riwayat input sebelumnya.

Ketika RNN di-unroll sepanjang dimensi waktu, neuron yang sama akan muncul kembali di setiap time step, dengan bobot yang dibagi (shared weights). Hal ini memungkinkan model mempelajari pola temporal tanpa menambah jumlah parameter secara linear terhadap panjang urutan.

Secara matematis, satu layer RNN sederhana melibatkan:

matriks bobot
𝑊
𝑥
W
x
	​

 untuk koneksi dari input,

matriks bobot
𝑊
ℎ
W
h
	​

 untuk koneksi dari hidden state sebelumnya,

vektor bias
𝑏
b,

serta fungsi aktivasi
𝜙
ϕ, yang umumnya berupa tanh.

Jenis Hubungan Input–Output pada RNN

Berdasarkan bentuk input dan output yang dipetakan, RNN dapat digunakan dalam beberapa konfigurasi utama:

Sequence-to-Sequence
Baik input maupun output berbentuk urutan, misalnya pada pemodelan time series atau machine translation.

Sequence-to-Vector
Sebuah urutan dipetakan menjadi satu vektor representasi, contohnya pada klasifikasi sentimen.

Vector-to-Sequence
Satu vektor digunakan untuk menghasilkan urutan, seperti pada tugas image captioning.

Encoder–Decoder
Urutan sumber dienkode menjadi representasi vektor oleh encoder, lalu decoder menghasilkan urutan target. Arsitektur ini umum digunakan pada penerjemahan mesin dan sequence transduction lainnya.

In [ ]:
import numpy as np
from tensorflow import keras

def generate_time_series(batch_size, n_steps):
    freq1, freq2, offs1, offs2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time - offs1) * (freq1 * 10 + 10))
    series += 0.2 * np.sin((time - offs2) * (freq2 * 20 + 20))
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)
    return series[..., np.newaxis].astype(np.float32)

n_steps = 50
series = generate_time_series(10000, n_steps + 1)
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]

model = keras.models.Sequential([
    keras.layers.SimpleRNN(1, input_shape=[None, 1])
])
model.compile(loss="mse", optimizer="adam")
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))


2. Training RNNs and Time Series Forecasting

RNN dilatih menggunakan Backpropagation Through Time (BPTT), yaitu dengan melakukan unrolling jaringan sepanjang beberapa langkah waktu. Pada tahap forward pass, model menghitung output dan nilai loss di setiap time step. Selanjutnya, pada backward pass, gradien disebarkan mundur melalui dimensi waktu.

Karena bobot yang sama digunakan berulang pada setiap langkah waktu, gradien untuk satu parameter akan dijumlahkan sepanjang urutan. Kondisi ini membuat RNN sangat rentan terhadap masalah vanishing gradients dan exploding gradients, terutama ketika menangani urutan yang panjang.

In [ ]:
from tensorflow import keras
import numpy as np

# One-step-ahead deep RNN with Dense output
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20),
    keras.layers.Dense(1)
])
model.compile(loss="mse", optimizer="adam")
model.fit(X_train, y_train, epochs=20,
          validation_data=(X_valid, y_valid))

# Forecast 10 steps ahead at once (sequence-to-vector)
series = generate_time_series(10000, n_steps + 10)
X_train, Y_train = series[:7000, :n_steps], series[:7000, -10:, 0]
X_valid, Y_valid = series[7000:9000, :n_steps], series[7000:9000, -10:, 0]

model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20),
    keras.layers.Dense(10)
])
model.compile(loss="mse", optimizer="adam")
model.fit(X_train, Y_train, epochs=20,
          validation_data=(X_valid, Y_valid))


3. Fighting Unstable Gradients and Using LSTM/GRU

RNN memiliki dua keterbatasan utama, yaitu gradien yang tidak stabil serta kemampuan memori jangka pendek. Untuk mengatasi masalah gradien, beberapa teknik yang umum digunakan pada deep feedforward networks tetap relevan, seperti:

optimizer berbasis momentum,

gradient clipping,

dropout,

serta inisialisasi bobot yang tepat.

Namun, Batch Normalization cenderung kurang efektif ketika diterapkan langsung di dalam sel RNN karena adanya ketergantungan antar time step.

Layer Normalization

Sebagai alternatif, Layer Normalization lebih cocok untuk arsitektur rekuren. Teknik ini menormalkan aktivasi di sepanjang dimensi fitur untuk setiap instance dan setiap langkah waktu, sehingga:

distribusi aktivasi menjadi lebih stabil,

proses pelatihan menjadi lebih konsisten,

dan sensitivitas terhadap inisialisasi bobot berkurang.

LSTM dan GRU

Untuk mengatasi keterbatasan memori jangka pendek pada RNN klasik, diperkenalkan dua arsitektur utama:

Long Short-Term Memory (LSTM)

Gated Recurrent Unit (GRU)

Kedua model ini menggunakan mekanisme gating—seperti forget, input, output, serta update/reset gate—dan mempertahankan state jangka panjang. Mekanisme ini memungkinkan jaringan:

menyimpan informasi penting,

melupakan informasi yang tidak relevan,

dan mempertahankan dependensi hingga puluhan langkah waktu.

Dalam praktiknya, LSTM dan GRU jauh lebih stabil dan mudah dilatih dibandingkan SimpleRNN, terutama pada urutan yang panjang. Oleh karena itu, keduanya sering menjadi pilihan default untuk berbagai tugas pemodelan sekuens.

In [ ]:
from tensorflow import keras

# Stacked LSTM seq2seq
model_lstm = keras.models.Sequential([
    keras.layers.LSTM(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.LSTM(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

# GRU with Conv1D front-end (see next section for context)
model_gru = keras.models.Sequential([
    keras.layers.GRU(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.GRU(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

# Custom SimpleRNNCell with Layer Normalization
class LNSimpleRNNCell(keras.layers.Layer):
    def __init__(self, units, activation="tanh", **kwargs):
        super().__init__(**kwargs)
        self.state_size = units
        self.output_size = units
        self.simple_rnn_cell = keras.layers.SimpleRNNCell(units, activation=None)
        self.layer_norm = keras.layers.LayerNormalization()
        self.activation = keras.activations.get(activation)

    def call(self, inputs, states):
        outputs, _ = self.simple_rnn_cell(inputs, states)
        norm_outputs = self.activation(self.layer_norm(outputs))
        return norm_outputs, [norm_outputs]

model_ln = keras.models.Sequential([
    keras.layers.RNN(LNSimpleRNNCell(20), return_sequences=True,
                     input_shape=[None, 1]),
    keras.layers.RNN(LNSimpleRNNCell(20), return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

4. Mixing RNNs with 1D Convolutions and WaveNet

Untuk urutan yang sangat panjang, bab ini mengeksplorasi kombinasi Conv1D dan RNN, atau bahkan mengganti RNN sepenuhnya dengan CNN satu dimensi. Conv1D dengan stride > 1 atau padding valid dapat melakukan downsampling temporal, sehingga memperpendek urutan sambil tetap menangkap pola lokal.

Dengan memilih kernel size dan stride yang tepat, model dapat melatih GRU atau LSTM di atas representasi urutan yang lebih ringkas. Output multi-step kemudian dihasilkan menggunakan TimeDistributed(Dense), yang terbukti meningkatkan performa forecasting.

Selain itu, diperkenalkan WaveNet, arsitektur berbasis CNN murni dengan dilated convolutions yang dilation-nya meningkat secara eksponensial (1, 2, 4, 8, …). Pendekatan ini memungkinkan model menangkap ketergantungan jangka sangat panjang secara efisien dan mencapai performa tinggi pada tugas audio dan sekuens panjang.

In [ ]:

from tensorflow import keras

# Conv1D front-end + GRU stack
model = keras.models.Sequential([
    keras.layers.Conv1D(
        filters=20, kernel_size=4, strides=2, padding="valid",
        input_shape=[None, 1]
    ),
    keras.layers.GRU(20, return_sequences=True),
    keras.layers.GRU(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

model.compile(loss="mse", optimizer="adam", metrics=[last_time_step_mse])
history = model.fit(
    X_train, Y_train[:, 3::2], epochs=20,
    validation_data=(X_valid, Y_valid[:, 3::2])
)

# Simplified WaveNet-style dilated Conv1D stack
wavenet = keras.models.Sequential()
wavenet.add(keras.layers.InputLayer(input_shape=[None, 1]))
for rate in (1, 2, 4, 8) * 2:
    wavenet.add(keras.layers.Conv1D(
        filters=20, kernel_size=2, padding="causal",
        activation="relu", dilation_rate=rate
    ))
wavenet.add(keras.layers.Conv1D(filters=10, kernel_size=1))
wavenet.compile(loss="mse", optimizer="adam", metrics=[last_time_step_mse])
history_w = wavenet.fit(
    X_train, Y_train, epochs=20,
    validation_data=(X_valid, Y_valid)
)

5. Practical Forecasting Patterns and Multi-Step Strategies

Bab ini menutup dengan beberapa strategi praktis untuk multi-step forecasting:

Iterative strategy
Model dilatih untuk memprediksi satu langkah ke depan, lalu prediksi tersebut digunakan kembali sebagai input untuk langkah berikutnya. Metode ini sederhana namun rentan terhadap akumulasi error.

Direct strategy
Model dilatih untuk langsung memprediksi beberapa langkah ke depan sekaligus sebagai output vektor. Pendekatan ini lebih stabil, meskipun bisa lebih sulit untuk horizon yang sangat jauh.

Sequence-to-sequence multi-horizon
Target diubah menjadi urutan vektor multi-step, sehingga setiap time step memprediksi beberapa langkah ke depan. Dengan RNN seq-to-seq dan TimeDistributed(Dense), sinyal loss menjadi lebih kaya, yang sering mempercepat dan menstabilkan training.

Strategi-strategi ini dikombinasikan dengan baseline yang kuat, pilihan arsitektur (SimpleRNN, LSTM, GRU, Conv1D, WaveNet), serta teknik stabilisasi seperti clipping, normalisasi, dan dropout untuk membangun model forecasting yang benar-benar kompetitif.

In [ ]:
import numpy as np
from tensorflow import keras

n_steps = 50
series = generate_time_series(10000, n_steps + 10)

Y = np.empty((10000, n_steps, 10))
for step_ahead in range(1, 11):
    Y[:, :, step_ahead - 1] = series[:, step_ahead: step_ahead + n_steps, 0]

Y_train, Y_valid, Y_test = Y[:7000], Y[7000:9000], Y[9000:]

model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

model.compile(loss="mse", optimizer="adam")
history = model.fit(X_train, Y_train, epochs=20,
                    validation_data=(X_valid, Y_valid))

